<a href="https://colab.research.google.com/github/vvikasreddy/lexically_constrained_beam_search_/blob/main/constraints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
from collections import defaultdict

In [ ]:
from datasets import load_dataset

ds = load_dataset("wmt/wmt16", "tr-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def get_ngrams(src, n = 2, ):
  src = src.split(" ")

  src = [tuple(src[i:i+n]) for i in range(len(src) - n + 1)]

  return src

In [ ]:
def constraints():

  dict_pairs = defaultdict(int)
  src_ngrams_count = defaultdict(int)
  tgt_ngrams_count = defaultdict(int)

  total_src_words = 0
  total_tgt_words = 0

  count = 0
  for doc in ds["train"]["translation"]:

    # get the src, and tgt
    src = doc["tr"]
    tgt = doc["en"]

    # get the ngrams, default n = 2
    src_ngrams = get_ngrams(src)
    tgt_ngrams = get_ngrams(tgt)


    # count the occurences of each of the ngram
    for tgt_ngram in tgt_ngrams:
      total_tgt_words += 1
      tgt_ngrams_count[tgt_ngram] += 1

    for src_ngram in src_ngrams:
      total_src_words += 1
      src_ngrams_count[src_ngram] += 1

    # # count the combined co-occurence of the ngram

    # for src_ngram in src_ngrams:
    #   for tgt_ngram in tgt_ngrams:
    #     dict_pairs[src_ngram, tgt_ngram] += min(src_ngrams_count[src_ngram], tgt_ngrams_count[tgt_ngram])

    if count %1000 == 0:
      print(count)
    count += 1

  return dict_pairs, src_ngrams_count, tgt_ngrams_count, total_src_words, total_tgt_words

# dict_pairs, src_ngrams_count, tgt_ngrams_count =  constraints()

In [ ]:
# min_count = 5
# filtered_src = {k: v for k, v in src_ngrams_count.items() if v >= min_count}
# filtered_tgt = {k: v for k, v in tgt_ngrams_count.items() if v >= min_count}

In [ ]:
def get_count_dict_pairs(filtered_src, filtered_tgt, src_ngrams_count, tgt_ngrams_count):

  dict_pairs = defaultdict(int)

  count = 0
  for doc in ds["train"]["translation"]:

    # get the src, and tgt
    src = doc["tr"]
    tgt = doc["en"]

    # get the ngrams, default n = 2
    src_ngrams = get_ngrams(src)
    tgt_ngrams = get_ngrams(tgt)

    # # count the combined co-occurence of the ngram

    for src_ngram in src_ngrams:
      if src_ngram not in filtered_src: continue
      for tgt_ngram in tgt_ngrams:

        if tgt_ngram not in filtered_tgt: continue
        dict_pairs[(src_ngram, tgt_ngram)] += min(src_ngrams_count[src_ngram], tgt_ngrams_count[tgt_ngram])

    if count %1000 == 0:
      print(count)
    count += 1

  return dict_pairs
# dict_pairs =  get_count_dict_pairs()

In [ ]:
# len(dict_pairs)
# import gc

In [ ]:
# dict_pairs

In [ ]:
# cleaning the unnecessary details



In [ ]:
import math

def calculate_pmi(dict_pairs, src_ngrams_count, tgt_ngrams_count, total_tgt_words, total_src_words):
    ignored_count = 0
    pmi_dict = {}
    count = 0

    total_dict_pairs = sum(dict_pairs.values())

    for (src_ngram, tgt_ngram), count in dict_pairs.items():
        p_src_tgt = dict_pairs[(src_ngram, tgt_ngram)] / total_dict_pairs

        p_src = src_ngrams_count[src_ngram] / total_src_words
        p_tgt = tgt_ngrams_count[tgt_ngram] / total_tgt_words

        if p_src == 0 or p_tgt == 0:
          pmi = 0
        else:
          pmi = math.log2(p_src_tgt / (p_src * p_tgt))/ -math.log2(p_src_tgt)


        if pmi > 0.9:
          pmi_dict[(src_ngram, tgt_ngram)] = pmi
        else:
          ignored_count += 1

    return pmi_dict


In [ ]:
def get_constraints():

  # get the count and filtering if occurences is less than min_count, set 5
  dict_pairs, src_ngrams_count, tgt_ngrams_count, total_src_words, total_tgt_words =  constraints()
  min_count = 3
  filtered_src = {k: v for k, v in src_ngrams_count.items() if v >= min_count}
  filtered_tgt = {k: v for k, v in tgt_ngrams_count.items() if v >= min_count}

  dict_pairs =  get_count_dict_pairs(src_ngrams_count=src_ngrams_count, tgt_ngrams_count= tgt_ngrams_count, filtered_src=filtered_src, filtered_tgt=filtered_tgt)


  pmi_scores = calculate_pmi(dict_pairs, filtered_src, filtered_tgt, total_tgt_words, total_src_words)


  # dictionary to only return the pair with max PMI value.
  max_pmi_dict = {}


  for (src, tgt), pmi_value in pmi_scores.items():
      if src not in max_pmi_dict:
          max_pmi_dict[src] = (tgt, pmi_value)
      else:
          if pmi_value > max_pmi_dict[src][1]:
              max_pmi_dict[src] = (tgt, pmi_value)

  return max_pmi_dict

In [ ]:
pmi_scores = get_constraints()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [ ]:
len(pmi_scores)

55

In [ ]:
max(pmi_scores.values())

(('well', 'as'), 1.3045270862505716)

In [ ]:
pmi_scores

{('Southeast', 'European'): (('Southeast', 'European'), 2.007429209652565),
 ('European', 'Times'): (('European', 'Times'), 1.9167591854827242),
 ('Times', 'için'): (('for', 'Southeast'), 1.8911110573345382),
 ('için', "Priştine'den"): (('in', 'Pristina'), 0.9216987197497427),
 ('haberi', '--'): (('for', 'Southeast'), 1.3272785799552846),
 ("SETimes'a", 'konuşan'): (('told', 'SETimes.'), 0.9561859994381473),
 ('için', "Üsküp'ten"): (('in', 'Skopje'), 0.9333156901837453),
 ("SETimes'a", 'verdiği'): (('told', 'SETimes.'), 0.9616320325311761),
 ('verdiği', 'demeçte,'): (('told', 'SETimes.'), 0.9982304560834028),
 ('Salı', 'günü'): (('on', 'Tuesday'), 1.1367580877880021),
 ('Dışişleri', 'Bakanı'): (('Foreign', 'Minister'), 1.370151811801477),
 ('Hırvatistan', 've'): (('Croatia', 'and'), 0.9307553543356438),
 ('yanı', 'sıra'): (('well', 'as'), 1.3045270862505716),
 ('AB', 've'): (('EU', 'and'), 1.0022170935178725),
 ('[Getty', 'Images]'): (('[Getty', 'Images]'), 1.5597082338725858),
 ('ve',